In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from dataclasses import dataclass, field
import os,sys,inspect
from transformers import (
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    glue_compute_metrics,
    glue_output_modes,
    glue_tasks_num_labels,
    set_seed,
)
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, EvalPrediction, GlueDataset
from transformers.data.processors.utils import InputFeatures
import torch
from torch.utils.data import Dataset, DataLoader
from typing import Dict, Optional
from sklearn.model_selection import train_test_split
from multiprocessing import Pool

import logging

In [3]:
logger = logging.getLogger(__name__)

In [4]:
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [5]:
import config

In [6]:
MODEL_NAME=config.PRE_TRAINED_MODEL_NAME

### Setup Path and load data

In [7]:
path = Path('/data/yelp/')
assert path.exists()

In [8]:
balanced_simplified_reviews = pd.read_pickle(path/'balanced_simplified_reviews_4m.pkl')
balanced_simplified_reviews['stars'] = balanced_simplified_reviews['stars'] - 1

In [9]:
balanced_simplified_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1570016 entries, 31 to 3999998
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   text    1570016 non-null  object
 1   stars   1570016 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 35.9+ MB


In [10]:
len(balanced_simplified_reviews)

1570016

In [11]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )

In [12]:
model_args = ModelArguments(
    model_name_or_path=MODEL_NAME
)

In [13]:
training_args = TrainingArguments(
    output_dir=str(path/'simple_output'),
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    per_gpu_train_batch_size=16,
    per_gpu_eval_batch_size=16,
    num_train_epochs=2,
    logging_steps=20000,
    logging_first_step=True,
    save_steps=40000,
    evaluate_during_training=True,
    fp16=True,
    eval_steps=60000
)

In [14]:
# ??TrainingArguments

In [15]:
set_seed(training_args.seed)

In [16]:
num_labels = config.NUM_CLASSES
num_labels

5

#### Prepare Tokenizer and Model

In [17]:
bert_config = AutoConfig.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels,
    finetuning_task='yelp_simple',
)

In [18]:
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME
)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    config=bert_config
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [21]:
# ??tokenizer.encode_plus

#### Prepare the Yelp dataset

In [22]:
class YelpDataset(Dataset):
    def __init__(self, reviews, targets, tokenizer, max_len):
        self.reviews, self.targets,  self.tokenizer, self.max_len = reviews.to_numpy(), targets.to_numpy(), tokenizer, max_len
        
    def __len__(self):
        return len(self.reviews)
        
    def __getitem__(self, item):
        review = self.reviews[item]
        tokens = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return InputFeatures(input_ids = tokens['input_ids'].flatten().long().numpy().tolist(), 
                             attention_mask=tokens['attention_mask'].flatten().long().numpy().tolist(),
                             label=torch.tensor(self.targets[item]).long().item())
    
    def get_labels(self):
        return ['1', '2', '3', '4', '5']

In [23]:
train_df, val_df = train_test_split(balanced_simplified_reviews, test_size=0.1, random_state=config.RANDOM_SEED, 
                                     stratify=balanced_simplified_reviews.stars.values)

In [24]:
def create_dataset(df, tokenizer, max_length, batch_size):
    return YelpDataset(df['text'], df['stars'], tokenizer, max_length)

In [25]:
train_dataset = create_dataset(train_df, tokenizer, config.MAX_LENGTH, 4)
eval_dataset = create_dataset(val_df, tokenizer, config.MAX_LENGTH, 4)

#### Metrics

In [26]:
def compute_metrics(p: EvalPrediction) -> Dict:
    preds = np.argmax(p.predictions, axis=1)
    return glue_compute_metrics('mnli', preds, p.label_ids)

#### Trainer and training

In [27]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [28]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


/opt/conda/envs/huggingface/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/envs/huggingface/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:114: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/opt/conda/envs/huggingface/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gat

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 65536.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 65536.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 65536.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Gradient overflow.  Skipping step, loss sc

/opt/conda/envs/huggingface/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)





CPU times: user 11h 33min 25s, sys: 3h 29min 11s, total: 15h 2min 37s
Wall time: 10h 17min 21s


TrainOutput(global_step=88314, training_loss=0.6752940556399956)

#### Evaluate

In [29]:
result = trainer.evaluate()

output_eval_file = os.path.join(training_args.output_dir, "eval_results.txt")
if trainer.is_world_master():
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results *****")
        for key, value in result.items():
            logger.info("  %s = %s", key, value)
            writer.write("%s = %s\n" % (key, value))

Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.


In [39]:
!echo {training_args.output_dir}
!cat {training_args.output_dir}/eval_results.txt

/data/yelp/simple_output
eval_loss = 0.6503205843860466
eval_mnli/acc = 0.7196150367511241
epoch = 2.0


#### Save the model

In [30]:
trainer.save_model()
# For convenience, we also re-save the tokenizer to the same directory,
# so that you can share your model easily on huggingface.co/models =)
if trainer.is_world_master():
    tokenizer.save_pretrained(training_args.output_dir)

#### Predict

Trying some batch predictions here

In [31]:
sample_df = val_df[:20]

In [32]:
predict_ds = create_dataset(sample_df, tokenizer, config.MAX_LENGTH, 4)

In [33]:
predictions, label_ids, metrics = trainer.predict(predict_ds)

Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.


In [34]:
predictions

array([[-4.328125  , -2.4902344 ,  1.0654297 ,  3.3144531 ,  2.4902344 ],
       [-5.6640625 , -4.0625    ,  0.9892578 ,  4.7070312 ,  3.9941406 ],
       [-2.5820312 ,  1.0205078 ,  3.0292969 ,  1.1171875 , -2.4726562 ],
       [-2.7089844 , -3.0683594 , -1.5800781 ,  1.4003906 ,  5.5898438 ],
       [-2.1367188 ,  0.1427002 ,  1.6035156 ,  1.2119141 , -0.99853516],
       [ 1.1445312 ,  3.6347656 ,  2.6328125 , -2.1542969 , -5.3476562 ],
       [-3.7421875 , -4.1796875 , -2.0214844 ,  3.1113281 ,  6.4765625 ],
       [-0.5151367 ,  2.5585938 ,  2.9765625 , -0.7128906 , -4.3789062 ],
       [-4.8828125 , -2.2441406 ,  1.4541016 ,  3.4296875 ,  2.3320312 ],
       [ 2.8710938 ,  4.1132812 ,  1.5527344 , -3.2265625 , -5.4375    ],
       [ 0.57128906,  4.171875  ,  3.1542969 , -2.2949219 , -5.65625   ],
       [ 7.0039062 ,  0.5317383 , -1.8134766 , -3.3515625 , -2.6230469 ],
       [ 3.5058594 ,  3.2324219 ,  0.5107422 , -3.3007812 , -4.3359375 ],
       [ 4.7226562 ,  3.5136719 , -0.1

In [35]:
preds = np.argmax(predictions, axis=1)

In [36]:
sample_df['preds'] =  preds 

/opt/conda/envs/huggingface/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [37]:
correct = np.sum((sample_df['stars'] == sample_df['preds']).astype('uint16'))
print(f'{correct} answers correct')
print(f'{correct / len(sample_df)} accuracy')

14 answers correct
0.7 accuracy


### Upload the model to the repository

In [46]:
final_dir_name = "/data/yelp/english-yelp-sentiment"

In [41]:
!echo {training_args.output_dir}
!mv {training_args.output_dir} {final_dir_name}

/data/yelp/simple_output


In [47]:
!ls {final_dir_name}

checkpoint-40000  eval_results.txt	   tokenizer_config.json
checkpoint-80000  pytorch_model.bin	   training_args.bin
config.json	  special_tokens_map.json  vocab.txt


In [45]:
from transformers import TFBertForSequenceClassification

In [49]:
tf_model = TFBertForSequenceClassification.from_pretrained(final_dir_name, from_pt=True)
tf_model.save_pretrained(final_dir_name)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [50]:
!ls {final_dir_name}

checkpoint-40000  eval_results.txt	   tf_model.h5		  vocab.txt
checkpoint-80000  pytorch_model.bin	   tokenizer_config.json
config.json	  special_tokens_map.json  training_args.bin


In [52]:
!rm -rf {final_dir_name}/checkpoint-40000 {final_dir_name}/checkpoint-80000 

In [53]:
!ls {final_dir_name}

config.json	   special_tokens_map.json  training_args.bin
eval_results.txt   tf_model.h5		    vocab.txt
pytorch_model.bin  tokenizer_config.json
